# 2 Session : POD

## Exercise 1: Face images

The goal of the exercise is to perform POD on a dataset containing face images
The dataset contains 400 64x64 images of human faces.
How can POD help in reconstructing faces?

In the first cell we import the libraries that we are going to use and the face dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_olivetti_faces

face_images = fetch_olivetti_faces().images
print(face_images.shape) #check the dimensions


We can also plot same faces.

In [ ]:
# select the index of the face to show
index = 0

# select the image and reshape it to show it
face = face_images[index]
plt.title(f'Face number = {index}')
plt.imshow(face, cmap='grey')
plt.xticks(np.linspace(0, 63, 8))
plt.yticks(np.linspace(0, 63, 8))

plt.show()

 - For convenience, please define the number of images of faces as **Nfaces** and the number of pixeles as **Npix**

 - To be able to apply POD, we need a matrix. 
     Please reshape the tensor face_images as a matrix (it will be our "X") and call it **face_X**

In [ ]:
Nfaces  = face_images.shape[0]
Npix1   = face_images.shape[1]
Npix    = face_images.shape[1]*face_images.shape[2]

print(f'Nfaces = {Nfaces}')
print(f'Npix   = {Npix}')

face_X = np.reshape(face_images, (Nfaces, Npix)).T
print(face_X.shape)

##  Implement POD
Now, we write a function to **implement the POD algorithm**. Conventionally, the mean is removed from the dataset before applying POD. If this is not done, the mean will be the first PC.

 - HINT to define POD:
     1. center the matrix X
     2. compute the covariance matrix K = X0.T @ X0
     3. compute the eigendecomposition using np.linalg.eig
     4. sort the eigenvalues using np.sort and np.argsort
     5. compute the POD modes Z = X0 @ Psi
     6. normalize the POD modes phi[i] = z[i]/np.linalg.norm(z[i])

In [ ]:
# define the POD function

def POD(X):
    X0 = X - np.mean(X, axis=1)[:, np.newaxis] # remove the mean from the dataset
    K = X0.T @ X0 # compute the covariance matrix
    l, Psi = np.linalg.eig(K) # compute the eigenvalue decomposition
    i_sort = np.argsort(l)[::-1] # sort the eigenvalues in descending order
    l = l[i_sort]
    Psi = Psi[:,i_sort]
    Z = X0 @ Psi  # Compute the PC scores
    
    Phi = np.zeros_like(Z)
    for i in range(Phi.shape[1]):
        Phi[:, i] = Z[:, i]/np.linalg.norm(Z[:, i])

    return Psi, Phi, l

# Uncomment when you have defined the function POD
Psi, Phi, l = POD(face_X)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.scatter(np.linspace(1, Nfaces, Nfaces), l)
ax.set_title('POD eigenvalues')
plt.xlim([0, Nfaces])
plt.xlabel('PC number')
plt.ylabel('Eigenvalues')
plt.show()


##  Compute POD using the SVD 
Now, we can perform POD using the SVD and compute the explained variance as a function of the modes.

In [ ]:
mean = np.mean(face_X, axis=1)[:, np.newaxis]
X0 = face_X - mean
Phi, sigma, Psi_t = np.linalg.svd(X0, full_matrices=False)
l = sigma**2

explained_variance_ratio = l/np.sum(l)*100

fig, axs = plt.subplots(1,2, figsize=(8,4))
axs[0].scatter(np.linspace(1, Nfaces, Nfaces), explained_variance_ratio)
axs[0].set_title('explained variance')
axs[1].plot(np.cumsum(explained_variance_ratio))
axs[1].set_title('cumulative explained variance')
plt.show()


 ##### Show the first eigenfaces.
 
- Plot the first n=5 eigenfaces.

In [ ]:
# We can plot the first 5 PCs 
n = 5
fig, axs = plt.subplots(1, n, figsize=(3*n, 3))
for i, ax in enumerate(axs):
    ax.imshow(Phi[:, i].reshape(Npix1, Npix1), cmap='grey')
    ax.set_title('Mode ' + str(i+1))


##### Test POD as a reduced-order model (ROM)
We can also check if we can use POD to achieve dimensionality reduction. This means that we can reconstructed the faces using few modes.

- To do:
    1. reconstruct the face number 10 using the first 10 modes, and compare it to the original
    2. reconstruct the same face with 1,5, 10, 100, 200 and 400 modes. Compare the solution
    
    
- #hint: X_reconstructed = mean + Phi_q Aq.T 
- #hint: Aq.T = Sigma_q @ Psi_q.T

In [ ]:
At = np.diag(sigma) @ Psi_t
A = At.T

index= 10
face_orig = face_X[:, index] # original face

fig1, ax = plt.subplots(figsize=(2, 2))
ax.imshow(face_orig.reshape(Npix1,Npix1), cmap='grey')
ax.set_title('original')
plt.show()

qs = [1, 5, 10, 100 , 200, 400]
Nq = len(qs)

# We can plot them side by side 
fig2, axs = plt.subplots(1,Nq, figsize=(3*Nq, 3)); 

for iq in range(0, Nq):  
    q=qs[iq]
    qperc = q/Nfaces *100
    face_rec = mean.flatten() + Phi[:, :q] @ A[index,:q].T        # reconstructed face
    
    axs[iq].imshow(face_rec.reshape(Npix1,Npix1), cmap='grey')
    axs[iq].set_title( str(q) + ' PCs (' + str(qperc) + '%)')
    
plt.show()

#### Canonical basis vs modes
Explain what is the difference between using the canonical basis and the modes to express the images, and why we can reduce the dimensionality with POD and not with the canonical basis. 

##### - Canonical basis
How does this work? In the canonical basis, the image is the sum of some weights multiplied by the canonical basis:
\begin{equation}
    \mathbf{x} = \sum_{i=1}^{m} w_i \mathbf{b}_i
\end{equation}

- To do: 
 1. Define the canonical basis $\mathbf{B}=[\mathbf{b}_1, \cdots \mathbf{b}_m]$
 2. Obtain the weights $w_i$ of the first 4 directions (n_basis=4)  $\mathbf{W}=[\mathbf{w}_1, \cdots \mathbf{w}_m]$
 3. Show how the first 4 basis vectors look like ($b_i$)

In [ ]:
index = 10
face = face_X[:, index] 

B = np.eye(Npix) # The canonical basis is just the identity matrix
n_basis = 4

## What are the weights? --> wi --> face_X[index,i]
w = face_X[:n_basis, index]

# Plotting function
fig, axs = plt.subplots(1, (n_basis+1), figsize=((n_basis+1)*3, 3))
fig.set_facecolor('white')
axs[0].imshow(face.reshape(Npix1,Npix1), cmap='grey')
axs[0].set_xticks([])
axs[0].set_yticks([])

for i in range(n_basis):
    axs[i+1].imshow(B[:, i].reshape(Npix1,Npix1), cmap='Greys')
    axs[i+1].yaxis.set_label_coords(-0.2,0.5)
    axs[i+1].set_xticks([])
    axs[i+1].set_yticks([])
    if i == 0:
        axs[i+1].set_ylabel('= {:.3f} $\cdot$'.format(w[i]), rotation=0, fontsize=14)
    else:
        axs[i+1].set_ylabel('+ {:.3f} $\cdot$'.format(w[i]), rotation=0, fontsize=14)

fig.subplots_adjust(left=0, right=1, bottom=0., top=1, wspace=0.4)
plt.show()

##    4. Plot original face VS reconstruction with 100 Modes
    
Obviously, we cannot truncate the canonical basis without loosing important information.

In [ ]:
q = 100    # number of modes to use
index = 10 # the index of the face that we want to reconstruct 

face_orig = face_X[:, index]                     # original face
face_rec  = face_X[:q, index] @ B[:,:q].T        # reconstructed face

# We can plot them side by side 
fig, axs = plt.subplots(1,2, figsize=(6, 3))
axs[0].imshow(face_orig.reshape(Npix1,Npix1), cmap='grey')
axs[0].set_title('original')
axs[1].imshow(face_rec.reshape(Npix1,Npix1), cmap='grey')
axs[1].set_title('reconstructed with ' + str(q) + ' PCs')
plt.show()

#### - POD basis
How does this work? In the POD basis, the image is the sum of some weights multiplied by the POD basis:
\begin{equation}
    \mathbf{x} = \mu(x) + \sum_{i=1}^{m} a_i \boldsymbol{\phi}_i
\end{equation}

- To do: 
 1. POD basis already defined: $\boldsymbol{\Phi}=[\boldsymbol{\phi}_1, \cdots \boldsymbol{\phi}_m]$
 2. POD coefficients already known too: $\mathbf{A}=[\mathbf{a}_1, \cdots \mathbf{a}_m]$ 
 3. Show how the first 4 basis vectors look like.

In [ ]:
index = 10
face  = face_X[:, index]

a = A[index, :]

fig, axs = plt.subplots(1, (n_basis+1), figsize = ((n_basis+1)*3,3) )

# plot first original face

axs[0].imshow(face.reshape(Npix1,Npix1), cmap = 'grey')
axs[0].set_xticks([])
axs[0].set_yticks([])

# plot b1
for i in range(n_basis):
    axs[i+1].imshow(Phi[:,i].reshape(Npix1,Npix1), cmap = 'grey')
    axs[i+1].yaxis.set_label_coords(-0.3,0.5)
    axs[i+1].set_xticks([])
    axs[i+1].set_yticks([])
    
    if i == 0:
        axs[i+1].set_ylabel('= {:.3f} $\cdots$'.format(a[i]),rotation= 0, fontsize=14)
    else:
        axs[i+1].set_ylabel('+ {:.3f} $\cdots$'.format(a[i]),rotation= 0, fontsize=14)

fig.subplots_adjust(left=0, right=1, bottom=0., top=1, wspace=0.6)      
plt.show()
